In [4]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os
import boto3

In [5]:


# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter variáveis de ambiente
MINIO_ENDPOINT = 'http://localhost:9000'
MINIO_ACCESS_KEY = os.getenv('AWSKEY')
MINIO_SECRET_KEY = os.getenv('AWSSECRETY')

In [6]:

# Configuração do SparkSession
spark = SparkSession.builder \
    .appName("Sales Data Ingestion") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.4") \
    .getOrCreate()

# Exemplo de leitura/escrita no MinIO
bucket_name = "my-bucket"
delta_path = f"s3a://{bucket_name}/delta/sales_data"

# Dados de exemplo
data = [("James", "Sales", 3000), ("Michael", "Sales", 4600), ("Robert", "Sales", 4100)]
columns = ["Name", "Department", "Salary"]

# Criação do DataFrame
df = spark.createDataFrame(data, columns)
df.show()



+-------+----------+------+
|   Name|Department|Salary|
+-------+----------+------+
|  James|     Sales|  3000|
|Michael|     Sales|  4600|
| Robert|     Sales|  4100|
+-------+----------+------+



In [8]:
# Criar o bucket se não existir (opcional)
import boto3
s3 = boto3.client('s3', endpoint_url='http://minio:9000', aws_access_key_id=MINIO_ACCESS_KEY, aws_secret_access_key=MINIO_SECRET_KEY)
s3.create_bucket(Bucket=bucket_name)




ClientError: An error occurred (InvalidAccessKeyId) when calling the CreateBucket operation: The Access Key Id you provided does not exist in our records.

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 42088)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [ ]:
# Escrevendo dados para o Delta Lake
df.write.format("delta").mode("overwrite").save(delta_path)

# Lendo dados do Delta Lake
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()